In [ ]:
!unzip "/content/Dataset 1.zip"

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import os
import cv2

# Function to load images and labels from the dataset folder
def load_dataset(folder_path):
    X = []  # List to store images
    Y = []  # List to store corresponding labels

    # Loop through each class in the dataset
    for class_name in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_name)

        # Ignore hidden files or directories (those starting with '.')
        if not class_name.startswith('.'):
            # Loop through each image in the class
            for filename in os.listdir(class_path):
                if not filename.startswith('.'):
                    image_path = os.path.join(class_path, filename)

                    # Load image using OpenCV or any other image loading library
                    image = cv2.imread(image_path)

                    # Preprocess your images if needed
                    # Example: Resize images to a specific size (e.g., 224x224) and normalize
                    image = cv2.resize(image, (224, 224))
                    image = image.astype('float') / 255.0  # Normalize to the range [0, 1]

                    # Append image and label to X and Y
                    X.append(image)
                    Y.append(class_name)

    return X, Y

# Define the path to your dataset
data_path = "/content/Dataset 1/Colorectal Cancer "

# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]),
}

# Create a single DataLoader for the entire dataset
dataset = datasets.ImageFolder(root=data_path, transform=data_transforms['train'])
dataloaders = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

# Check if GPU is available, otherwise use CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load pre-trained MobileNetV2 model
model = models.mobilenet_v2(pretrained=True)

# Modify the classifier for your specific classification task
model.classifier[1] = nn.Linear(1280, len(dataset.classes))  # Adjust to the number of classes

# Send the model to GPU if available
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10

for epoch in range(num_epochs):
    model.train()

    running_loss = 0.0
    corrects = 0

    for inputs, labels in dataloaders:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataset)
    epoch_acc = corrects.double() / len(dataset)

    print(f'Epoch {epoch + 1}/{num_epochs} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print("Training complete!")


Epoch 1/10 Loss: 0.1927 Acc: 0.9278
Epoch 2/10 Loss: 0.1084 Acc: 0.9645
Epoch 3/10 Loss: 0.0782 Acc: 0.9730
Epoch 4/10 Loss: 0.0750 Acc: 0.9757
Epoch 5/10 Loss: 0.0396 Acc: 0.9867
Epoch 6/10 Loss: 0.0765 Acc: 0.9737
Epoch 7/10 Loss: 0.0335 Acc: 0.9885
Epoch 8/10 Loss: 0.0253 Acc: 0.9905
Epoch 9/10 Loss: 0.0418 Acc: 0.9867
Epoch 10/10 Loss: 0.0359 Acc: 0.9883
Training complete!
